In [ ]:
import osmnx as ox
import geopandas as gpd
import folium

# Define TTDI bounding box
north, south, east, west = 3.1505, 3.128, 101.641, 101.615
bbox = (west, south, east, north)

# Fetch plantable areas from OSM
tags = {
    'leisure': 'park',
    'landuse': 'grass',
    'natural': 'grass'
}

try:
    plantable_areas = ox.features_from_bbox(bbox, tags)
    plantable_areas = plantable_areas.to_crs(epsg=3857)
    plantable_areas = plantable_areas[plantable_areas.geometry.notnull()]
    # Explode mixed geometries
    plantable_areas = plantable_areas.explode(index_parts=False).reset_index(drop=True)
    plantable_areas = plantable_areas[plantable_areas.geometry.type.isin(['Polygon', 'MultiPolygon'])]
    print(f"Plantable areas found: {len(plantable_areas)}")
except Exception as e:
    print(f"Error fetching plantable areas: {e}")
    plantable_areas = gpd.GeoDataFrame(columns=['geometry'], geometry=[])

# Fetch existing trees (points) from OSM
tree_tags = {'natural': 'tree'}
try:
    existing_trees = ox.features_from_bbox(bbox, tree_tags)
    existing_trees = existing_trees.to_crs(epsg=3857)
    existing_trees = existing_trees[existing_trees.geometry.notnull()]
    existing_trees = existing_trees[existing_trees.geometry.type == 'Point']
    print(f"Existing trees found: {len(existing_trees)}")
except Exception as e:
    print(f"Error fetching existing trees: {e}")
    existing_trees = gpd.GeoDataFrame(columns=['geometry'], geometry=[])

# Remove areas too close to existing trees
if not existing_trees.empty and not plantable_areas.empty:
    existing_trees['geometry'] = existing_trees.geometry.buffer(2)  # 2m buffer
    plantable_areas = gpd.overlay(plantable_areas, existing_trees, how='difference')
    print(f"Plantable areas after removing existing trees: {len(plantable_areas)}")

#  Function to suggest tree based on plot size (m²) an example
def suggest_tree(area_m2):
    if area_m2 < 5:
        return "Hibiscus"  # small shrub
    elif area_m2 < 20:
        return "Plumeria"
    elif area_m2 < 50:
        return "Frangipani"
    elif area_m2 < 100:
        return "Angsana"
    else:
        return "Rain Tree"  # large tropical tree

# Calculate area and assign tree
plantable_areas['area_m2'] = plantable_areas.geometry.area
plantable_areas['suggested_tree'] = plantable_areas['area_m2'].apply(suggest_tree)

# Convert to WGS84 for Folium
plantable_areas = plantable_areas.to_crs(epsg=4326)
center_lat = (north + south) / 2
center_lon = (east + west) / 2

# Create Folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=16, tiles='CartoDB positron')

# Convert buffered existing trees to WGS84 for Folium
if not existing_trees.empty:
    existing_trees = existing_trees.to_crs(epsg=4326)

# Shade plantable areas (blue)
for _, row in plantable_areas.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda feature, color="blue": {
            'fillColor': color,
            'color': 'darkblue',
            'weight': 1,
            'fillOpacity': 0.5
        },
        tooltip=f"Area: {row['area_m2']:.1f} m²\nSuggested tree: {row['suggested_tree']}"
    ).add_to(m)

# Shade existing trees (green)
for _, row in existing_trees.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda feature: {
            "fillColor": "green",
            "color": "green",
            "weight": 1,
            "fillOpacity": 0.5
        },
        tooltip="Existing tree"
    ).add_to(m)

# Save map
output_path = "/notebooks/ttdi_plantable_area.html"
m.save(output_path)
print(f"Map saved successfully at {output_path}")
m


Plantable areas found: 23
Existing trees found: 12
Plantable areas after removing existing trees: 23
Map saved successfully at /notebooks/ttdi_plantable_area.html
